In [1]:
%reset -f

In [2]:
import pacKage
import loaddata
import uNet
from pacKage import *
from uNet import *
from loaddata import *
rootPath = os.getcwd()
dataPath = os.path.join(rootPath,"data")
modelPath = os.path.join(rootPath,"model")
submitPath = os.path.join(rootPath,"submission")
if not os.path.exists(dataPath):
    os.makedirs(dataPath)
if not os.path.exists(modelPath):
    os.makedirs(modelPath)
if not os.path.exists(submitPath):
    os.makedirs(submitPath)
os.listdir(dataPath)

device is cuda


['label', 'test', 'train']

- load the dataset

In [4]:
test_dir  = os.path.join(dataPath, "test")

In [5]:
test_ince_rgb_dl_temp    =     TestDataset(test_dir = test_dir)
test_ince_rgb_dl         =     DataLoader(test_ince_rgb_dl_temp, batch_size = 8)

- load the model

In [6]:
incemodel = UShapeNet(4).to(device=device)  # Make sure to instantiate the model class first
incemodel.load_state_dict(torch.load(os.path.join(modelPath, "ince_model_params.pth")))
rgbmodel = UShapeNet(3).to(device=device)
rgbmodel.load_state_dict(torch.load(os.path.join(modelPath, "rgb_model_params.pth")))

<All keys matched successfully>

In [24]:
for index, batch in enumerate(iter(test_ince_rgb_dl)):
    ince, rgb, name = batch
    ince_perdiction, rgb_prediction = incemodel(ince).argmax(1), rgbmodel(rgb).argmax(1)
    prediction = ince_perdiction + rgb_prediction
    binary_prediction = (prediction == 2)
    
    binary_prediction = binary_prediction.int().detach().cpu().numpy()
    for i in range(prediction.shape[0]):
        tiff.tifffile.imwrite(os.path.join(submitPath , name[i]), binary_prediction)

('AA408972_satellite.tif', 'AA804258_satellite.tif', 'AB121622_satellite.tif', 'AB144957_satellite.tif', 'AB363647_satellite.tif', 'AB455766_satellite.tif', 'AC615524_satellite.tif', 'AC799254_satellite.tif')
<class 'tuple'>
('AD373903_satellite.tif', 'AD551860_satellite.tif')
<class 'tuple'>


- create the model to deal with our binary output in the shappe of (350, 350, 2) each from our incemodel and rgbmodel!!!